### RB SmartCard

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from os.path import join
from bs4 import BeautifulSoup
from importlib import reload
import codecs
import re

import roamability as rb

downloads = 'C:/Users/balob/Downloads'

rb.greetings_func()

Greetings from Roamability!!! 



In [104]:
reload(rb)

<module 'roamability' from 'C:\\Users\\balob\\Documents\\GITLAB\\RB_BD\\roamability.py'>

In [25]:
class SmartCard:
    
    def __init__(self, imsi, msisdn=None, vmsc=None, sgsn=None):
        self.imsi = imsi
        self.msisdn = msisdn
        self.vmsc = vmsc
        self.sgsn = sgsn
        self.ss7 = rb.SendMsgSs7()
        self.ogt = self.select_ogt(imsi)
        
    def select_ogt(self, imsi):
        if imsi.startswith('42501'):
            return self.ss7.ogt['partner']
        elif imsi.startswith('26006'):
            return self.ss7.ogt['p4']
        elif imsi.startswith('24040'):
            return self.ss7.ogt['netmore']
 
    def print_imsi_prof(self, payload_type='as_resp'):
        if self.imsi and self.vmsc:
            self.ss7.print_imsi_prof(self.ogt, self.imsi, self.vmsc, payload_type)
        else:
            print('Set IMSI and MSC GT')
            
    def get_config(self):
        if self.imsi and self.vmsc:
            resp = self.ss7.sendSMS(self.ogt, self.imsi, self.vmsc, self.ss7.get_config)
            print(resp.prettify, '\n')
            if resp.response.response:
                self.ss7.get_config_decoder(resp.response.response.text.upper()[-40:])
            else:
                print('No info in response.')
        else:
            print('Set IMSI and MSC GT')

In [16]:
# Send message
card = SmartCard(imsi='425019613960139', msisdn='', vmsc='79639743660', sgsn='79639743872')
# card.ss7.cl(card.ogt, card.vmsc, card.imsi, d_ssn=7)
# card.ss7.cl(card.ogt, card.sgsn, card.imsi, d_ssn=149)
# card.ss7.sendSMS(card.ogt, card.imsi, card.vmsc, card.ss7.get_info)

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result>
<src_gt>79639743872</src_gt>
</response>

In [3]:
# Print IMSI profiles
card = SmartCard(imsi='425019628115014', msisdn='', vmsc='46705012099', sgsn='')
card.print_imsi_prof()

<bound method Tag.prettify of <?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>40077FF644027100003F0ABFFF0100000000000000FFFFFFF00001080242F0100000003A25370586215905000829040400001100410108290660103461054102084952106982110541</response>
<src_gt>46705009041</src_gt>
</response>> 

This SIM card contains the following IMSIs:
Slot 0 240400000110014
Slot 1 260060143165014
Slot 2 425019628115014

Applet ver.: 1.8.2
MCCMNC: 24001


In [113]:
# Get card Config
card = SmartCard(imsi='260060148997960', msisdn='', vmsc='447802000130')
card.get_config()

<bound method Tag.prettify of <?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>00077FF62902710000240ABFFF01000000000000000000005B00010302001E001400000003000305010201000000</response>
<src_gt>447802000130</src_gt>
</response>> 

Applet enabling flag: Activate
Mode: Events & Timer mode
Default IMSI: IMSI2
Polling period: 30 sec.
Features activation flags: 0000000000010100
    Start with Primary: Disable
    Reset Loci, PS Loci: Disable
    RB_MODE (Sponsor IMSI in HPLMN): Enable
    Skip Primary IMSI in roaming: Disable
Refresh Type: 00
Default refresh Type: 00
No Service Swap Timer: 3 sec.
Refresh Retry: 05
Refresh Retry Delay: 1 sec.
Scan delay timer: 3 sec.


In [33]:
ss7 = rb.SendMsgSs7()
ss7.get_config_decoder('010301001E001000000003000305010202000000') # Netmore wrong

Applet enabling flag: Activate
Mode: Events & Timer mode
Default IMSI: IMSI1
Polling period: 30 sec.
Features activation flags: 0000000000010000
    Start with Primary: Disable
    Reset Loci, PS Loci: Disable
    RB_MODE (Sponsor IMSI in HPLMN): Disable
    Skip Primary IMSI in roaming: Disable
Refresh Type: 00
Default refresh Type: 00
No Service Swap Timer: 3 sec.
Refresh Retry: 05
Refresh Retry Delay: 1 sec.
Scan delay timer: 3 sec.


In [34]:
ss7.get_config_decoder('010302001E001400000003000305010201000000') # Netmore RAO from prod SIM card

Applet enabling flag: Activate
Mode: Events & Timer mode
Default IMSI: IMSI2
Polling period: 30 sec.
Features activation flags: 0000000000010100
    Start with Primary: Disable
    Reset Loci, PS Loci: Disable
    RB_MODE (Sponsor IMSI in HPLMN): Enable
    Skip Primary IMSI in roaming: Disable
Refresh Type: 00
Default refresh Type: 00
No Service Swap Timer: 3 sec.
Refresh Retry: 05
Refresh Retry Delay: 1 sec.
Scan delay timer: 3 sec.


In [16]:
# IMSI decoder

imsi = '4952106941005106'
imsi = '425019614001560'

def imsi_conv(imsi, code='code'):
    """Conver deconvert IMSI for perso. code='code' / 'decode'"""
    if code=='code':
        imsi = '9' + imsi
        return ''.join([imsi[i] for i in [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])
    elif code=='decode':
        return ''.join([imsi[i] for i in [0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])

# imsi_conv(imsi, code='decode')
imsi_conv(imsi, code='code')

'4952106941005106'

In [17]:
# ANY decoder

value = '4952106941005106'

def any_decoder(value):
    value_conv = ''
    for i in range(1, len(value), 2):
        value_conv += value[i: i + 1]
        value_conv += value[i - 1: i]
    return value_conv

any_decoder(value)

'9425019614001560'

In [22]:
# ASCII decoder

value = b"Smartel"
value = '524F414D4142494C49' # 'ROAMABILI'

def ascii_conv(value, code='code'):
    if code=='code':
        return codecs.encode(value, "hex").upper()
    elif code=='decode':
        return bytearray.fromhex(value).decode()

# ascii_conv(value, code='code')
ascii_conv(value, code='decode')

'ROAMABILI'

In [40]:
l = len('89 97 21 91 21 00 00 46 00 00 89 97 21 91 21 00 00 55 99 9F 4E A4 72 13 A4 A9 47 5B') / 2
print('Dec to hex')
print(f'{int(l)} -> 0x{int(l):X}')

Dec to hex
41 -> 0x29


In [ ]:
ota_env =\
"""
D1 51 82 02  83 81 06 04   01 99 99 99  8B 45 40 06   
01 88 88 88  7F F6 02 70   20 22 23 11  30 35 02 70   
00 00 30 15  16 01 19 19   B0 00 11 55  2C 28 B7 60   
04 80 D0 33  56 08 78 29   5A F4 7B 5C  12 A1 7A 00   
73 9B 60 A3  CB A0 58 4F   B5 03 61 B9  6C 38 B7 47   
F0 13 17 
"""

def ota_cmd(ota_env):
    ota_env = ota_env.replace(' ', '').replace('\n', '')
    print(f'{ota_env}')
    print(f'send_apdu -APDU A0C20000{int(len(ota_env)/2):X}{ota_env}')

ota_cmd(ota_env)

### Developments

In [9]:
class TestClass:
    c_var01 = 'TestClassVar'
    def __init__(self, i_var=None):
        self.i_var01 = i_var
        self.i_var02 = self.c_var01

test_class = TestClass()

In [10]:
TestClass.c_var01 = 'TestClassVar01'
test_class.i_var02
test_class.i_var01

'TestInstanceVar'

In [24]:
# Send message

payload = '027000001F0D00000000BFFF010000000000002028D0FC11B4CD2E540704D7775B065DA8'

card = SmartCard(imsi='425019613000536', msisdn='', vmsc='79219600320', sgsn='79219600113')
card.ss7.sendSMS(card.ogt, card.imsi, card.vmsc, card.ss7.get_info)

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>00077FF65802710000530ABFFF0100000000000000FFFFFFF0000108000000000000008A567809437451000008495210693100506301082906601094005063020829430500620400100308495210697100000004082906306601000040</response>
<src_gt>79219600320</src_gt>
</response>

In [7]:
# Netmore Test

# Print IMSI profiles
card = SmartCard(imsi='425019628115014', msisdn='', vmsc='46705012099', sgsn='')
card.print_imsi_prof()

<bound method Tag.prettify of <?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>40077FF644027100003F0ABFFF0100000000000000FFFFFFF00001080242F0100000003A25370586215905000829040400001100410108290660105400511702084952106982110541</response>
<src_gt>46705009041</src_gt>
</response>> 

This SIM card contains the following IMSIs:
Slot 0 240400000110014
Slot 1 260060145001571
Slot 2 425019628115014

Applet ver.: 1.8.2
MCCMNC: 24001


In [6]:
# Change IMSI Slot1 260060143165014 -> 260060145001570

payload = '02700000370D00010000BFFF0100000000000010B6065E351FC9819DB107C4CE9AC48FD216F306C2B8F268F27451F1AB0BE9D40D3165850360EBF32A'

card.ss7.sendSMS(card.ogt, card.imsi, card.vmsc, payload)

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>40077FF61502710000100ABFFF01000000000000000000000100</response>
<src_gt>46705009041</src_gt>
</response>

In [31]:
# Print IMSI profiles
card = SmartCard(imsi='425019628115014', msisdn='46726900014', vmsc='46705012099', sgsn='')
# card = SmartCard(imsi='240400000110014', msisdn='46726900014', vmsc='46731726121', sgsn='')
card.print_imsi_prof()

<bound method Tag.prettify of <?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>Error: SM Delivery Failure(32)</error>
<src_gt>46705009041</src_gt>
</response>> 

No info in response.


In [27]:
card.ss7.sri4sm(card.ss7.ogt['netmore'], card.msisdn, card.msisdn)

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result>
<imsi>240400000110014</imsi>
<msc nat="1" nbp="1">46731726312</msc>
<src_gt>46731726121</src_gt>
</response>

In [37]:
s1 = '42F0044080'
s2 = '42F0044080'
s1 == s2

True

In [ ]:
01110011

In [39]:
rb.gsm_encode("sv")
# 7376656E64616E6F666964657275667265736974706C

b'7376'